# Arrays and Loops

We'll cover:
1. Array literals
2. Concatenation
3. For loops
4. Comprehensions
5. Element types
6. Dequeues
7. The `bang!` convention
8. Variable names vs. copies

# Arrays

Julia has highly efficient multidimensional arrays, both constructed and indexed with square brackets.

Syntax: <br>
```julia
[item1, item2, ...]
```

In [1]:
squares = [1, 4, 9, 15, 25, 36, 49, 64]

8-element Vector{Int64}:
  1
  4
  9
 15
 25
 36
 49
 64

In [2]:
squares[1]

1

In [3]:
squares[1:3]

3-element Vector{Int64}:
 1
 4
 9

In [4]:
squares[end]

64

In [5]:
squares[4] = 16

16

In [6]:
squares

8-element Vector{Int64}:
  1
  4
  9
 16
 25
 36
 49
 64

In [7]:
typeof(squares)

Vector{Int64} (alias for Array{Int64, 1})

## Concatenation

If, instead of commas, you just use spaces, then the values are concatenated horizontally.

In [8]:
cubes = [1, 8, 27, 64, 125, 216, 343, 512]

8-element Vector{Int64}:
   1
   8
  27
  64
 125
 216
 343
 512

In [9]:
powers = [1:8 squares cubes]

8×3 Matrix{Int64}:
 1   1    1
 2   4    8
 3   9   27
 4  16   64
 5  25  125
 6  36  216
 7  49  343
 8  64  512

In [10]:
powers[4, 2]

16

In [11]:
powers[:, 3]

8-element Vector{Int64}:
   1
   8
  27
  64
 125
 216
 343
 512

In [12]:
powers[7, :]

3-element Vector{Int64}:
   7
  49
 343

In [13]:
typeof(powers)

Matrix{Int64} (alias for Array{Int64, 2})

Semicolon separators perform vertical concatenation:

In [14]:
[squares; cubes]

16-element Vector{Int64}:
   1
   4
   9
  16
  25
  36
  49
  64
   1
   8
  27
  64
 125
 216
 343
 512

Whereas commas would simply create an array of arrays:

In [15]:
nested_powers = [[1,2,3,4,5,6,7,8], squares, cubes]

3-element Vector{Vector{Int64}}:
 [1, 2, 3, 4, 5, 6, 7, 8]
 [1, 4, 9, 16, 25, 36, 49, 64]
 [1, 8, 27, 64, 125, 216, 343, 512]

In [16]:
nested_powers[2]

8-element Vector{Int64}:
  1
  4
  9
 16
 25
 36
 49
 64

Horizontal and vertical concatenation can be used together to as a simple syntax for matrix literals:

In [17]:
[1 3 5; 2 4 6]

2×3 Matrix{Int64}:
 1  3  5
 2  4  6

# Loops

Of course, we could construct this programmatically with a for-loop. The syntax for a `for` loop is

```julia
for *var* in *loop iterable*
    *loop body*
end
```



In [18]:
A = fill(0, (8, 3)) # Allocate an 8x3 matrix to store the values into
for pow in 1:3
    for value in 1:8
        A[value, pow] = value ^ pow
    end
end
A

8×3 Matrix{Int64}:
 1   1    1
 2   4    8
 3   9   27
 4  16   64
 5  25  125
 6  36  216
 7  49  343
 8  64  512

In [19]:
A == powers

true

## Array Comprehensions

In [20]:
squares = [value^2 for value in 1:8]

8-element Vector{Int64}:
  1
  4
  9
 16
 25
 36
 49
 64

In [21]:
cubes = [value^3 for value in 1:8]

8-element Vector{Int64}:
   1
   8
  27
  64
 125
 216
 343
 512

In [22]:
powers = [value^pow for value in 1:8, pow in 1:3]

8×3 Matrix{Int64}:
 1   1    1
 2   4    8
 3   9   27
 4  16   64
 5  25  125
 6  36  216
 7  49  343
 8  64  512

# The element type

Note that every time an array prints out, it is displaying its element type and dimensionality, for example `Array{Int64, 2}`. This describes what it can store — and thus what it can return upon indexing.

In [23]:
typeof(powers)

Matrix{Int64} (alias for Array{Int64, 2})

In [24]:
typeof(powers[1, 1])

Int64

Further, the array will try to convert any new values assigned into it to its element type:

In [25]:
powers[1, 1] = 1.6

LoadError: InexactError: Int64(1.6)

In [27]:
powers[1, 1] = -5.0 # This can be losslessly converted to an integer

-5.0

In [28]:
powers

8×3 Matrix{Int64}:
 -5   1    1
  2   4    8
  3   9   27
  4  16   64
  5  25  125
  6  36  216
  7  49  343
  8  64  512

Arrays that have an exact and concrete element type are generally significantly faster, so Julia will try to find an amenable element type for you in its literal construction syntax:

In [29]:
fortytwosarray = [42, 42.0, 4.20e1, 4.20f1, 84//2, 0x2a]

6-element Vector{Float64}:
 42.0
 42.0
 42.0
 42.0
 42.0
 42.0

In [30]:
for x in fortytwosarray
    show(x)
    println("\tisa $(typeof(x))")
end

42.0	isa Float64
42.0	isa Float64
42.0	isa Float64
42.0	isa Float64
42.0	isa Float64
42.0	isa Float64


The `Any` array can be helpful for disabling these behaviors and allowing all kinds of different objects:

In [31]:
anyfortytwos = Any[42, 42.0, 4.20e1, 4.20f1, 84//2, 0x2a]

6-element Vector{Any}:
   42
   42.0
   42.0
   42.0f0
  42//1
 0x2a

In [32]:
anyfortytwos[1] = "FORTY TWO"
anyfortytwos

6-element Vector{Any}:
     "FORTY TWO"
   42.0
   42.0
   42.0f0
  42//1
 0x2a

# Vectors as dequeues

One-dimensional arrays can be appended to and have items removed from them:

In [63]:
fib = [1, 1, 2, 3, 5, 8, 13]

7-element Vector{Int64}:
  1
  1
  2
  3
  5
  8
 13

In [64]:
push!(fib, 21)

8-element Vector{Int64}:
  1
  1
  2
  3
  5
  8
 13
 21

In [65]:
push!(fib, sum(fib[end-1:end]))

9-element Vector{Int64}:
  1
  1
  2
  3
  5
  8
 13
 21
 34

In [66]:
pop!(fib)

34

In [67]:
pushfirst!(fib, 0)

9-element Vector{Int64}:
  0
  1
  1
  2
  3
  5
  8
 13
 21

In [68]:
popfirst!(fib)

0

## Aside: why so shouty!?

Why are there exclamations in all the above function calls? `push!(fib, ...)`, `pop!(fib)`, etc.?

In [39]:
push!

push! (generic function with 25 methods)

This is entirely a convention. It's a signal to the caller that one of the argument is going to be _mutated_. It's perhaps easiest to demonstrate with an example:

In [69]:
A = rand(0:10, 10)

10-element Vector{Int64}:
  8
  2
  9
  4
  4
  3
 10
  9
  1
  1

In [70]:
sort(A)

10-element Vector{Int64}:
  1
  1
  2
  3
  4
  4
  8
  9
  9
 10

In [71]:
A

10-element Vector{Int64}:
  8
  2
  9
  4
  4
  3
 10
  9
  1
  1

In [72]:
sort!(A)

10-element Vector{Int64}:
  1
  1
  2
  3
  4
  4
  8
  9
  9
 10

In [73]:
A # That changed A!

10-element Vector{Int64}:
  1
  1
  2
  3
  4
  4
  8
  9
  9
 10

## Aside: names vs. copies

Remember that variables are just names we give our objects. So watch what happens if we give the same object two different names:

In [74]:
fibonacci = [1, 1, 2, 3, 5, 8, 13]

7-element Vector{Int64}:
  1
  1
  2
  3
  5
  8
 13

In [75]:
some_numbers = fibonacci
some_numbers[1] = 404
some_numbers

7-element Vector{Int64}:
 404
   1
   2
   3
   5
   8
  13

In [76]:
fibonacci

7-element Vector{Int64}:
 404
   1
   2
   3
   5
   8
  13

In [77]:
fibonacci[1] = 1
some_numbers = copy(fibonacci)
some_numbers[2] = 404
fibonacci

7-element Vector{Int64}:
  1
  1
  2
  3
  5
  8
 13